<a href="https://colab.research.google.com/github/REU-DS-CLUB/RecSys-films/blob/master/artem_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
#!pip install gdown

In [249]:
!pip install --upgrade --no-cache-dir gdown

In [250]:
import gdown
import pandas as pd
import numpy as np
import random

In [251]:
!gdown --id 1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
!gdown --id 1LggfAKKnsJ1iCOWQI7XdVsSRE-R0CLsJ
!gdown --id 1WiW7K8F-E8afXoeqyMalkOlfYATRuS9V
!gdown --id 1ILq5E5i4P6kneSziQwXP_bDCc2B7A7RE
!gdown --id 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown --id 1XO_bjqQc6Tq6jQdfas5hq0t2jTyjBaz1
!gdown --id 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
To: /content/credits.csv
100% 190M/190M [00:01<00:00, 189MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1LggfAKKnsJ1iCOWQI7XdVsSRE-R0CLsJ
To: /content/keywords.csv
100% 6.23M/6.23M [00:00<00:00, 148MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https:

**movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**keywords.csv:** Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

**credits.csv:** Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

**links.csv:** The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

**links_small.csv:** Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

**ratings.csv:** Ratings of the Full Dataset.

**ratings_small.csv:** The subset of 100,000 ratings from 700 users on 9,000 movies.

In [252]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [253]:
movies_metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


**1 Получим случайную матрицу эмбедингов для каждого фильма**

In [254]:
# из исходных данных беру айдишник и добавляю 30 случайных колонок
matrix_emb = pd.concat([movies_metadata[['id']], 
                        pd.DataFrame(np.random.random((movies_metadata.shape[0], 30)))], 
                       axis=1)

Удаляем очевидные ошибки в данных и меняем тип колонки id на int, чтобы мэтчился с другой таблицей

In [255]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '1997-08-20'].index)

In [256]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '2012-09-29'].index)

In [257]:
matrix_emb = matrix_emb.drop(matrix_emb[matrix_emb.id == '2014-01-01'].index)

In [258]:
matrix_emb['id'] = matrix_emb['id'].astype(int)

**2 Теперь напишем функцию train_test_split по юзеру по матрице рейтингов (юзер х фильмы = рейтинг)**

**Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях**


In [259]:
ratings = pd.read_csv('ratings.csv')

In [260]:
ratings.sample(5)

,userId,movieId,rating,timestamp
14474569,150432,593,3.5,1222031102
11037742,114167,3176,3.5,1118169470
25836856,268793,2433,3.0,955649297
20909498,217287,2841,4.0,955643368
14989752,155918,1844,4.0,956855741


In [261]:
def train_test_split_films(dataframe, user, size=0.7):

  '''
     на вход подается название датафрейма с рейтингами, 
     айди нужного пользователя и пропорция разделения

     эта функция выведет данные именно для одного пользователя
     можно доработать для большего количества, добавив цикл по
     передаваемому спику айдишников

     на выхлде получаем словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tt_data_dict = {}
  user_tt_data_list = []

  # выбираю данные по нужному юзеру и перемешиваю
  temp_df = dataframe[dataframe['userId'] == user] \
            .sample(dataframe[dataframe['userId'] == user].shape[0]) \
            [['movieId', 'rating']]

  # разбиваю их на трейн и тест в заданной пропроции
  train = temp_df.iloc[0:int(temp_df.shape[0]*size)]
  test = temp_df.iloc[int(temp_df.shape[0]*size):]
  
  # разделяю каждый набор на фильмы и рейтинги и записываю в общий список
  user_tt_data_list.append(train['movieId'].tolist())
  user_tt_data_list.append(train['rating'].tolist())
  user_tt_data_list.append(test['movieId'].tolist())
  user_tt_data_list.append(test['rating'].tolist())

  # список со всеми векторами записываю в словарь под ключ - айди зрителя
  user_tt_data_dict[user] = user_tt_data_list

  return user_tt_data_dict

In [262]:
splitted_data = train_test_split_films(ratings, 270896, 0.7)

**3 Для каждого юзера по train-фильмам находим средний вектор эмбединг через mean**

**Надо дополнительно домножить эмбединг-вектор на оценку фильма**

In [263]:
def get_avg_weightened_vec(dict_name, user_id, movies_emb_df):

  '''
     на вход подается название словаря с данными после сплита,
     айдишник нужного зрителя и название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор
  '''

  # находим все вектора фильмов из train-выборки в эмбединг-матрице
  # и джойним с их рейтингами от выбранного пользователя
  temp_df = movies_emb_df \
            .loc[movies_emb_df['id'].isin(dict_name[user_id][0])] \
            .merge(pd.DataFrame(dict_name[user_id][0], dict_name[user_id][1], 
                                columns=['id']) \
                                .reset_index() \
                                .rename(columns={'index':'rating'}), on='id')

  # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
  avg_weightened_vec = list(temp_df \
                            .drop(['id'], axis=1)
                            .mul(temp_df['rating'], axis=0)
                            .drop(['rating'], axis=1)
                            .mean())

  return avg_weightened_vec

In [264]:
res_vec = get_avg_weightened_vec(splitted_data, 270896, matrix_emb)

In [265]:
res_vec

[2.003849913823004,
 1.9453068279955958,
 2.1828456227737956,
 1.9213305200368027,
 1.8850296946889844,
 2.0780737041790824,
 2.029628639235709,
 1.9896511755302984,
 2.0755977567380874,
 1.9959027677709429,
 2.065895728219533,
 2.1618889206078675,
 1.9737276079916972,
 1.9675260749936572,
 2.116293483409847,
 2.1792608618659814,
 1.8455905175323428,
 1.8805682834070616,
 1.9772073041765317,
 1.8284024698335413,
 1.8843946273287624,
 1.7673322367249351,
 2.1577701022883407,
 1.9507894398127699,
 2.097024893555933,
 1.8982855980948585,
 2.0792678833314633,
 1.8087732305814843,
 1.9272308019187079,
 2.026644844189946]